# Modules EXPERT

Esta tabla contiene la información a nivel de fichero (módulo).

## 1 - Obtencion de datos

In [ ]:
import matplotlib.pyplot as plt
from notebooks_utils import *
from domains import *

TARGET_FEATURE = 'module__expertise_level'
TARGET_CLASS = 'EXPERT'
full_table = get_data('modules')

instances_for_class_low = len(full_table[full_table[TARGET_FEATURE] == "BEGINNER"])
instances_for_class_high = len(full_table[full_table[TARGET_FEATURE] == "EXPERT"])

print(f"The dataset contains {instances_for_class_low} ({instances_for_class_low/len(full_table)*100:.4}%) instances for BEGINNER class and {instances_for_class_high} ({instances_for_class_high/len(full_table)*100:.4}%) for EXPERT class.")

In [ ]:
full_table = full_table[full_table[TARGET_FEATURE] == TARGET_CLASS]

# Table name, features and target.
TABLE_NAME = 'modules'
TABLE_FEATURES = ['module__name_convention', 'module__has_doc_string', 'module__global_stmts_pct', 'module__global_expressions_pct', 'module__number_of_classes', 'module__number_of_functions', 'module__class_defs_pct', 'module__function_defs_pct', 'module__enum_defs_pct', 'module__average_stmts_function_body', 'module__average_stmts_method_body', 'module__type_annotations_pct', 'module__has_entry_point']

# Load features and target.
X = full_table[TABLE_FEATURES]

# Print information about the loaded table.
print(f' {TARGET_CLASS} CLASS Features shape: {X.shape}')

print(f'As we can see the downloaded data contains a total of {X.shape[0]} instances of {TARGET_CLASS}. Each of the instances corresponds to a file. For each file we have {X.shape[1]} attributes.')

## 2 - Exploracion de datos

Una vez tenemos nuestra tabla en un dataframe el siguiente paso es explorarla para ver qué tipo de información contiene.

In [ ]:
print(X.info())

Cómo podemos ver la tabla está compuesta por diez variables numéricas, dos variables boleanas y una de tipo objeto. Las dos últimas variables son la etiqueta de la instancia y el name convention.

### 2.1 - Valores Nulos 
Miramos si alguna de las variables que contiene la tabla contiene algún valor que sea nulo.

In [ ]:
X.isnull().sum()

In [ ]:
print_empty_cols(X)

### 2.2 - Describimos los valores de las variables de la tabla.

In [ ]:
np.transpose(X.describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int']))

Vamos a discretizar las variables numericas, agrupando conjuntos de valores en categorias, para hacer un análisis de los datos. Para cada variable es necesaario ver la distribucion de lo valores para hacer los bins (categorias).

In [ ]:
# DISCRETIZATION
X_copy = X.copy()

discretized_columns = {
    "module__class_defs_pct": [(0.0, 0.0),(0.0, 1.0), (1.0, 1.0)],  # min: 0.0 max: 1.0
    "module__function_defs_pct": [(0.0, 0.0),(0.0, 1.0), (1.0, 1.0)],  # min: 0.0 max: 1.0
    "module__enum_defs_pct": [(0.0, 0.0), (0.0, inf)],  # min: 0.0 max: 0.0
    "module__number_of_classes": [(0, 0), (1, 1), (1,inf)],  # min: 0 max: 113
    "module__number_of_functions": [(0, 0), (0, inf)], # min: 0 max: 258
    "module__type_annotations_pct": [(0.0, 0.0), (0.0, inf)], # min: 0.0 max: 1.0
    "module__average_stmts_function_body": [(0.0, 0.0), (0.0, inf)], #min: 0.0 max: 53.0
    "module__average_stmts_method_body": [(0.0, 0.0), (0.0, inf)], # min: 0.0 max: 114.0
    "module__global_expressions_pct": [(0,0), (0.0, inf)], # min: 0.0 max: 1.0
    "module__global_stmts_pct": [(0, 0), (0.0, inf)] # min: 0.0 max: 1.0
}

discretize_columns(X_copy, discretized_columns)
    
# SINGLE FEATURE
print("--- SINGLE FEATURE ---")
print(get_statistics(X_copy, ['module__class_defs_pct'], 10))
print(get_statistics(X_copy, ['module__function_defs_pct'], 10))
print(get_statistics(X_copy, ['module__enum_defs_pct'], 10))
print(get_statistics(X_copy, ['module__number_of_classes'], 10))
print(get_statistics(X_copy, ['module__number_of_functions'], 10))
print(get_statistics(X_copy, ['module__type_annotations_pct'], 10))
print(get_statistics(X_copy, ['module__average_stmts_function_body'], 10))
print(get_statistics(X_copy, ['module__average_stmts_method_body'], 10))
print(get_statistics(X_copy, ['module__global_expressions_pct'], 10))
print(get_statistics(X_copy, ['module__global_stmts_pct'], 10))
print(get_statistics(X_copy, ['module__has_doc_string'], 10))
print(get_statistics(X_copy, ['module__has_entry_point'], 10))
print(get_statistics(X_copy, ['module__name_convention'], 10))


# 2 FEATURES
print("--- TWO FEATURES ---")
print(get_statistics(X_copy, ['module__function_defs_pct', 'module__number_of_classes'], 10))

# 3 FEATURES
print("--- THREE FEATURES ---")
print(get_statistics(X_copy, ['module__class_defs_pct', 'module__function_defs_pct', 'module__enum_defs_pct'], 10))

In [ ]:
def normalize_datatypes(X:pd.DataFrame) -> (pd.DataFrame, [str]):
    X = pd.get_dummies(X)
    X = X.astype('float32')        
    columns_names = X.columns.tolist()
    return X, columns_names

X, TABLE_FEATURES = normalize_datatypes(X)
# Print information about the loaded table
print(f'Features shape: {X.shape}')

#### Muestra la matriz de correlación de Spearman entre las variables de la tabla.

In [ ]:
sns.heatmap(X.corr(method='spearman'), annot=False)

## 3 - Detección de valores atípicos (outliers)
## Univariate
## Analisis detallado de variables
Para cada una de las 13 variable (2 booleanas, 6 porcentuales, 4 numericas y una categorica) se hara un analisis detallado

### Variable has_doc_string (1/13)
Esta variable es de tipo booleano y representa si el fichero contiene un comentario como primer nodo.

In [ ]:
plt.xticks([0, 1], ['False', 'True'])
sns.histplot(X['module__has_doc_string'], discrete=True)

In [ ]:
num_true_values = len(X[X['module__has_doc_string'] == True])
num_false_values = len(X[X['module__has_doc_string'] == False])
std = X['module__has_doc_string'].std()
freq_true = len(X[X['module__has_doc_string'] == True]) / len(X['module__has_doc_string'])
freq_false = len(X[X['module__has_doc_string'] == False]) / len(X['module__has_doc_string'])
print(f'La variable has_doc_string contiene {num_true_values} valores verdaderos y {num_false_values} valores falsos. Y, la desviación típica es de {std}')
print(f'La frecuencia con la que programa puede tener la variable has_doc_string a True es {freq_true}.')
print(f'La frecuencia con la que programa puede tener la variable has_doc_string a False es {freq_false}.')

In [ ]:
possible_values = [True, False]
print_frequency_anal_for_cat_var(X, 'module__has_doc_string', possible_values)

In [ ]:
X[X['module__has_doc_string'] == True].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])


### Variable has_entry_point (2/13)
Esta variable es de tipo booleano y representa si el fichero contiene el idiom "if __name__ == '__main__'". 

In [ ]:
plt.xticks([0, 1], ['False', 'True'])
sns.histplot(X['module__has_entry_point'], discrete=True)

In [ ]:
#ANALISIS DE DISTRIBUCIÓN DE DATOS
num_true_values = len(X[X['module__has_entry_point'] == True])
num_false_values = len(X[X['module__has_entry_point'] == False])
std = X['module__has_entry_point'].std()
freq_true = len(X[X['module__has_entry_point'] == True]) / len(X['module__has_entry_point'])
freq_false = len(X[X['module__has_entry_point'] == False]) / len(X['module__has_entry_point'])
print(f'La variable has_entry_point contiene {num_true_values} valores verdaderos y {num_false_values} valores falsos. Y, la desviación típica es de {std}')
print(f'La frecuencia con la que programa puede tener la variable has_entry_point a True es {freq_true}.')
print(f'La frecuencia con la que programa puede tener la variable has_entry_point a False es {freq_false}.')

In [ ]:
#PROBABILIDAD DE VALORES
possible_values = [True, False]
print_frequency_anal_for_cat_var(X, 'module__has_entry_point', possible_values)

In [ ]:
X[X['module__has_entry_point'] == True].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### Variable number_of_classes (3/13)
Esta variable es de tipo numérica y nos indica el número de clases definidos en el fichero. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 113. Con una media de 1.09.

In [ ]:
sns.stripplot(X['module__number_of_classes'])

### IMPORTANTE
Los tres valores máximos están muy alejados del siguiente.

In [ ]:
print_outliers_for_df_column(X, 'module__number_of_classes')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 73.08 (Con un MC de 96 usamos U)

In [ ]:
X[X['module__number_of_classes'] > 73.08].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE

Hay tres modulos que tienen un número de clases muy superior al resto.

### Variable number_of_functions (4/13)
Esta variable representa el número de funciones en el fichero. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 258. Con una media de 2.23.

In [ ]:
sns.stripplot(X['module__number_of_functions'])

### IMPORTANTE
Los dos valores máximos están muy alejados del siguiente.

In [ ]:
print_outliers_for_df_column(X, 'module__number_of_functions')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 143 (Con un MC de 96 usamos U)

In [ ]:
X[X['module__number_of_functions'] > 143].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE

Hay dos modulos que tienen un número de funciones muy superior al resto.

### Variable average_stmts_function_body (5/13)
Esta variable representa el número medio de sentencias en el cuerpo de las funciones del fichero. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 53. Con una media de 2.52.

In [ ]:
sns.stripplot(X['module__average_stmts_function_body'])

### IMPORTANTE
Los dos valores máximos están muy alejados del siguiente.

In [ ]:
print_outliers_for_df_column(X, 'module__average_stmts_function_body')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 41.9 (Con un MC de 46 usamos U)

In [ ]:
X[X['module__average_stmts_function_body'] > 41.9].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
Hay tres modulos que tienen un número de sentencias en el cuerpo de las funciones superior al resto.

### Variable average_stmts_method_body (6/13)
Esta variable representa el número medio de sentencias en el cuerpo de los métodos del fichero. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 114. Con una media de 2.02.

In [ ]:
sns.stripplot(X['module__average_stmts_method_body'])

### IMPORTANTE
Los dos valores máximos están muy alejados del siguiente.

In [ ]:
print_outliers_for_df_column(X, 'module__average_stmts_method_body')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 59.0 (Con un MC de 59 usamos U)

In [ ]:
X[X['module__average_stmts_method_body'] > 59.0].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
Hay un modulo que tiene un número de sentencias en el cuerpo de los métodos superior al resto.

### Variable name_convention (7/13)
Esta variable es de tipo categórica y representa la name_convention del nombre del fichero.

In [ ]:
print_categorical_histogram(full_table, 'module__name_convention')

In [ ]:
print_frequency_anal_for_cat_var(full_table, 'module__name_convention')

In [ ]:
print_values_usage_for_cat_var(full_table, 'module__name_convention', NAMING_CONVENTION_VALUES)

### Variable global_stmts_pct (8/13)
Esta variable representa la proporción de los nodos en el global scope que son sentencias. Toma los valores de 0.0 - 1.0. La media es 0.16.

In [ ]:
sns.displot(X['module__global_stmts_pct'], bins=[i / 100 for i in range(0, 101)])

In [ ]:
print_outliers_for_df_column(X, 'module__global_stmts_pct')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 0.625 (Con un MC de 53 usamos Tuckey)

In [ ]:
X[X['module__global_stmts_pct'] > 0.625].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
El 64% de los modulos no tiene sentencias en el global scope.

### Variable global_expressions_pct (9/13)
Esta variable representa la proporción de los nodos en el global scope son expresiones. Este campo toma valores del 0.0 - 1.0. Su media es 0.08.

In [ ]:
sns.displot(X['module__global_expressions_pct'], bins=[i / 100 for i in range(0, 101)])

In [ ]:
print_outliers_for_df_column(X, 'module__global_expressions_pct')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 0.36 (Con un MC de 82 usamos Tuckey Extremo)

In [ ]:
X[X['module__global_expressions_pct'] > 0.37].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE

 El 60% de los modulos no tiene expresiones en el global scope.


### Variable class_defs_pct (10/13)
Esta variable representa la proporción de definiciones que son definiciones de clases. Esta variable toma valores de 0.0 - 1.0. Su media es 0.39. 

In [ ]:
sns.displot(X['module__class_defs_pct'], bins=[i / 100 for i in range(0, 101)])

In [ ]:
print_outliers_for_df_column(X, 'module__class_defs_pct')

### IMPORTANTE
Estos valores siguen un distribucion en forma de U (distribucion de Laplace). Con lo que los posibles valores outliers estan en el medio, no en los extremos. No se puede aplicar el metodo de tukey. En este momento, solo vamos a mostrar los valores que hay en el medio.

In [ ]:
X[(X['module__class_defs_pct'] > 0.0) & (X['module__class_defs_pct'] < 1.0)].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE

 El 50% de los modulos no tiene clases definidas.

### Variable function_defs_pct (11/13)
Esta variable representa la proporción de deficiones que son definiciones de funciones. Esta variable toma valores de 0.0 - 1.0. Su media es 0.40

In [ ]:
sns.displot(X['module__function_defs_pct'], bins=[i / 100 for i in range(0, 101)])

In [ ]:
print_outliers_for_df_column(X, 'module__function_defs_pct')

### IMPORTANTE
Estos valores siguen un distribucion en forma de U (distribucion de Laplace). Con lo que los posibles valores outliers estan en el medio, no en los extremos. No se puede aplicar el metodo de tukey. En este momento, solo vamos a mostrar los valores que hay en el medio.

In [ ]:
X[(X['module__function_defs_pct'] > 0.0) & (X['module__function_defs_pct'] < 1.0)].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE

 Casi el 50% de los modulos no tienen definiciones de funciones

### Variable enum_defs_pct (12/13)
Esta variable representa la proporción de las definiciones que son clases con la notación enum. Esta variable toma valores de 0.0 - 0.0. Su media es 0.05.

In [ ]:
sns.displot(X['module__enum_defs_pct'], bins=[i / 100 for i in range(0, 101)])

In [ ]:
print_outliers_for_df_column(X, 'module__enum_defs_pct')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 0 (Todos los umbrales son 0)

In [ ]:
X[X['module__enum_defs_pct'] > 0.0].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
Apenas un 2% de los modulos tienen definiciones de enums.

### Variable type_annotations_pct (13/13)
Esta variable representa la proporción de los parámetros y returns de funciones y métodos que tienen una anotación de tipo. Toma valores entre 0.0 - 0.90. Su media es 0.19.

In [ ]:
sns.displot(X['module__type_annotations_pct'], bins=[i / 100 for i in range(0, 101)])

In [ ]:
print_outliers_for_df_column(X, 'module__type_annotations_pct')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 0 (Todos los umbrales son 0)

In [ ]:
X[X['module__type_annotations_pct'] > 0.0].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
El 40% de los modulos tienen anotaciones de tipo.

## Multivariate

Isolation forest algorithm

In [ ]:
from sklearn.ensemble import  IsolationForest

CONTAMINATION_FACTOR = 0.0012
isof_model = IsolationForest(contamination=CONTAMINATION_FACTOR, random_state=0)
isof_prediction = isof_model.fit_predict(X.values)
mask = isof_prediction == -1
full_table.loc[X.index[mask]]

### IMPORTANTE
Analizar los 14 resultados obtenidos.

In [ ]:
# Outliers
np.transpose(X[mask].describe(percentiles=[.25, .50, .75], include=['object', 'float', 'bool', 'int']))

In [ ]:
# Non-Outliers
np.transpose(X[~mask].describe(percentiles=[.25, .50, .75], include=['object', 'float', 'bool', 'int']))